# Rancher Based Modeling

In [15]:
import pandas as pd
from IPython.display import Image

For the final project in Space-Time Analytics I will be adding to an Agent Based Model of rancher stocking response to drought. The program Agent Analyst is used as an interface between the ABM program Repast and ArcGIS. It allows Repast to access data in the GIS, create agents and environments out of that, apply action and scheduling code that alters the data, then overwrite the data in the GIS and update the display for a continuous presentation of the phenomena that the ABM is simulating. The phenomenon we are looking at is the adoption of drought insurance by ranchers in response to drought. This is a precipitation-index based insurance product made available through the USDA's Risk Management Agency last year as means of supporting feed costs and mitigating against financial loss from lost grassland forage potential and reductions in cattle stock. The decision to purchase insurance will be based partly on simulated behaviors (that can verified or updated with actual data in the future) and actual data derived from historical precipitation records and cattle sale data from the last 10 years. The decision to purchase the insurance is considered to be based on need, with the largest indicator of such, in the data available to us, at least, being cattle sales. It is expected that there is a relationship between rainfall and cattle sales, with rising sales in response to dry conditions. This relationship will be the foundation of a decision function that each simulated rancher will use to decide to adopt insurance, which, once purchased, will provide a dampening effect on the amount of cattle that must be sold. Other factors and the conceptual code are described below:

Each time-step (1 months), each polygon agent will:

-assess their average precipitation levels and compare it to the average baseline precipitation (the “strike-level” that triggers payout) to see if they would have been eligible for a payout;

If precip < strike: coulda_count = 1 coulda_count = coulda_count + 1

-look back and see how many times they could have had a payout and if it reaches a certain ratio we could add that as a decision weight for whether or not to buy insurance.

    If coulda_count / timestep_count > .5 (any threshold ratio):
        woulda_count = woulda_count + 1 ( adding to our weight counter)
        
-Assess what ratio of neighbors have taken out insurance this year, or still has insurance this year, to the overall number of neighbors. Then, if the ratio reaches a certain threshold we will add weight of neighborhood to the decision equation.

    If neighbors_did > .5 #(or any threshold ratio):
        shoulda_count = shoulda_count.timestep[:-2] + 1
(We could also base this on a ratio, as in the more neighbors who have it the more likely the rancher is to buy it)

-Assess the amount of precipitation, and adjust their stocking rate accordingly by selling cows, buying cows or doing nothing. If he has to sell off a certain ratio of his stock we will add a weight of lost revenue to the decision equation. We will have to find an equation that tells us how much cattle they would sell at each at each precipitation level (drought_effect). This may not be field, we will probably need to include the equation

    stock = stock + (roundup(stock * (drought_effect)/stock)
        If roundup(stock * (drought_effect)/stock  >  5 (a loss_tolerance):
            Stockweight_count = stockweight_count + 1
-If the agent has insurance, it will provide a dampening effect on the amount of stock he must sell, or might also add to how much he buys . If we have time we could use this to decide whether or not he would buy better insurance (higher strike-level and liability)

-Finally, take into account his situation, weigh his options (buy or not buy) and decide whether or not to do it. Also, maybe how much insurance to take out and at what strike level.

    Maybe_should = some function of (woulda_count,shoulda_count and Stockweight_count)

If maybe_should > thats_it: Self.insured = True

In [20]:
Image('images/Trial_run.gif',width = 400, height = 400)

ValueError: Cannot embed the 'gif' image format

Below is the table I would like to use for the ABM, it contains average precipitaion at each cattle sale zone for each
monthly interval

In [6]:
df = pd.read_csv('data/ranchers.csv')
df.head()

,FID,location,F1,F2,F3,F4,F5,F6,F7,F8,...,F124,F125,F126,F127,F128,F129,F130,RANCHER_ID,COLOR,PrecipAv
0,0,"Eastanollee, GA",86.71,225.99,262.60,99.63,104.90,61.62,125.59,166.01,...,106.23,111.01,128.26,186.59,99.81,75.34,92.03,0,GREEN,101.82
1,1,"Carrollton, GA",119.67,105.11,79.16,79.28,98.13,53.33,75.03,72.35,...,94.85,98.72,48.23,79.42,108.02,109.23,120.05,1,GREEN,96.12
2,2,"Greensboro, GA",105.75,116.64,83.71,80.62,87.32,135.00,90.02,36.66,...,82.33,104.19,89.30,75.69,71.81,97.57,135.48,2,GREEN,94.20
3,3,"Jackson, GA",125.05,59.53,96.11,73.40,127.73,153.99,160.75,138.75,...,74.24,76.16,50.72,69.00,92.11,81.41,125.61,3,GREEN,95.07
4,4,"Swainsboro, GA",106.83,79.37,88.31,157.49,135.15,33.33,97.03,73.49,...,112.63,78.05,90.51,55.28,52.82,39.79,48.65,4,GREEN,94.53


Below is the data frame I want to use to find any correlations between cattle sale head counts and precipitation. For a previous 
we tried to find a simple linear model fit for all of Georgia and found no relationship. I believe that I might find a trend if 
I run a regression for each of the polygons and it has been suggested to perform a log transformations and bin the precipitation
values into dry and wet ranges. I had expected not to find a linear relationship from the beginning because it didn't seem like cattle sales would be sensitive to increasingly wet conditions, only to the particularly dry ones. I also wonder if there might be a lagged effect, i.e. ranchers might not always rush straight to the market when their rain gauge indicates drought. There might not be such an effect, they might base their decisions on climate forecasts. 

In [7]:
df2 = pd.read_csv('data/Final_cows.csv')
df2.head()

,ID,location,interval,X,Y,count,precip
0,13,"Americus, GA",2015_09,-84.23,32.07,164.0,99.20
1,13,"Americus, GA",2015_08,-84.23,32.07,420.0,85.98
2,13,"Americus, GA",2015_07,-84.23,32.07,250.0,79.38
3,13,"Americus, GA",2015_06,-84.23,32.07,139.0,55.40
4,13,"Americus, GA",2015_05,-84.23,32.07,154.0,21.69
